In [ ]:
#default_exp data.datasets

In [ ]:
# hide
from nbdev.showdoc import *
from nbdev.export import *
from nbdev.imports import Config as NbdevConfig
from lightning_cv.utils.data import download_data, folder2df

nbdev_path = str(NbdevConfig().path("nbs_path")/'data')
nbdev_path

'/Users/ayushman/Desktop/lightning_cv/nbs/data'

In [ ]:
# export
import torch
from lightning_cv.utils.common import get_default_logger
from lightning_cv.utils.data import pil_loader, cv2_loader